In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os as os
import numpy as np
import pickle

import json
import warnings
from collections import OrderedDict

import xarray as xr
import holoviews as hv
import geoviews as gv
import iris
import cartopy

from cartopy import crs
from cartopy import feature as cf
from geoviews import feature as gf

hv.notebook_extension('bokeh','matplotlib')

geolocalized_bureaux = pd.read_csv('../../data/processed/geolocalized_bureaux.csv')

In [2]:
def sum_population(data):
    data_Gps = [data.longitude.iloc[0] , data.latitude.iloc[0]]
    name = data.locality.iloc[0]
    pop = sum(data.N_voters)
    out = pd.DataFrame([{'long':data_Gps[0] , 'lat':data_Gps[1] , 'name':name , 'N_voters':pop}])
    return out

pop = geolocalized_bureaux.groupby('renaloc_ID').apply(sum_population)
#pop.columns = ['renaloc_ID' , 'total_population']

In [3]:
pop = pop.reset_index()
pop = pop[['lat','long','name','N_voters']]

In [4]:
population = gv.Dataset(pop, kdims=['N_voters' , 'name'])

In [5]:
cities = population.to(gv.Points, kdims=['long', 'lat'],
                    vdims=['N_voters'  , 'name'], crs=crs.PlateCarree())

In [6]:
%%opts WMTS [width=450 height=250 xaxis=None yaxis=None]

from bokeh.models import WMTSTileSource
from bokeh.tile_providers import STAMEN_TONER

In [7]:
%%output backend='bokeh'
%%opts Overlay [width=800 height=600 xaxis=None yaxis=None] 
%%opts Points (size=0.1 cmap='inferno') [tools=['hover'] color_index=2]
gv.WMTS(WMTSTileSource(url='http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png')) * cities

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [long,lat]   (N_voters,name)